In [ ]:
#--------------------------------------------------------------------------
#   MODELO DE AUTOAPRENDIZAJE CON TAXI
#--------------------------------------------------------------------------
#-- Este código recolecta datos del entorno Taxi-v3 y entrena un modelo para predecir el siguiente estado,
#-- dado un estado actual y una acción. No se usan recompensas, por lo que es autoaprendizaje (self-supervised learning).
#--------------------------------------------------------------------------

#--------------------------------------------------------------------------
# 1. Importamos las librerías necesarias
#--------------------------------------------------------------------------
import gymnasium as gym
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

#--------------------------------------------------------------------------
# 2. Recolectamos datos del entorno Taxi-v3
#--------------------------------------------------------------------------
print( '1. Recolectamos datos del entorno de Taxi-v3' )
entorno = gym.make( "Taxi-v3" )

transitions = []
EPISODIOS = 500  # Número de episodios para recolectar datos (500)

for _ in tqdm(range(EPISODIOS)):
    estado, _ = entorno.reset()
    FIN = False
    while not FIN:
        action = entorno.action_space.sample()  # Elegimos una acción aleatoria (sin política)
        proximo_estado, _, FIN, _, _ = entorno.step(action)
        # Guardamos la transición (estado, acción, siguiente estado)
        transitions.append((estado, action, proximo_estado))
        estado = proximo_estado

#--------------------------------------------------------------------------
# 3. Preprocesamiento de datos
#--------------------------------------------------------------------------
print( '\n2. Hacemos el preprocesamiento de los datos del entorno de Taxi-v3' )
STATE_SIZE = entorno.observation_space.n
ACTION_SIZE = entorno.action_space.n

# One-hot encode states and actions
def one_hot(index, size):
    vec = np.zeros(size)
    vec[index] = 1
    return vec

X = []  # Entradas: estado + acción (one-hot)
y = []  # Salidas: siguiente estado (one-hot)

for s, a, s_next in transitions:
    state_vec = one_hot(s, STATE_SIZE)
    action_vec = one_hot(a, ACTION_SIZE)
    input_vec = np.concatenate([state_vec, action_vec])
    target_vec = one_hot(s_next, STATE_SIZE)
    X.append(input_vec)
    y.append(target_vec)

# Convertir a numpy.array antes de convertir en tensor
X = np.array(X, dtype=np.float32)  # Convertir la lista a un solo array de numpy
y = np.array(y, dtype=np.float32)  # Convertir la lista a un solo array de numpy

# Ahora convierte a tensor de PyTorch
X = torch.tensor(X)
y = torch.tensor(y)

#--------------------------------------------------------------------------
# 4. Definimos el modelo como una Red neuronal para predecir el siguiente estado
#--------------------------------------------------------------------------
print( '\n3. Definimos el modelo de red neuronal de datos del entorno de Taxi-v3' )
class StatePredictor(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, output_size),
            nn.Softmax(dim=1)  # Distribución sobre los estados posibles
        )

    def forward(self, x):
        return self.net(x)

MODELO = StatePredictor( STATE_SIZE + ACTION_SIZE, STATE_SIZE )
criterion = nn.CrossEntropyLoss()  # Usamos clasificación categórica (una clase verdadera)
optimizer = optim.Adam( MODELO.parameters(), lr = 0.01 ) # 0.001

#--------------------------------------------------------------------------
# 5. Realizamos el entrenamiento
#--------------------------------------------------------------------------
print( '\n4. Realizamos el entrenamiento con el modelo de datos del entorno de Taxi-v3' )
EPOCAS      = 10
BATCH_SIZE  = 128  #64

for epoch in range( EPOCAS ):
    permutation = torch.randperm(X.size(0))
    loss_total = 0
    for i in range(0, X.size(0), BATCH_SIZE):
        indices = permutation[i:i+BATCH_SIZE]
        batch_x, batch_y = X[indices], y[indices]

        preds = MODELO(batch_x)
        loss = criterion(preds, torch.argmax(batch_y, dim=1))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_total += loss.item()
    print(f"    Época {epoch+1}/{EPOCAS}, Pérdida: {loss_total:.4f}")

# Al finalizar, el modelo ha aprendido una función de transición aproximada:
# (estado, acción) → siguiente estado probable

#--------------------------------------------------------------------------
# 6. Evaluamos el modelo con predicciones
#--------------------------------------------------------------------------
print( '\n5. Mostramos la evaluación del modelo con prediccines de datos del entorno de Taxi-v3' )
print( "\    Ejemplo de predicciones del modelo:")

estadoReal = []
estadosPredichos = []
accionesTomadas = []

estado, _ = entorno.reset()
for _ in range( 5 ):
    accion = entorno.action_space.sample()
    input_vec = np.concatenate([one_hot(estado, STATE_SIZE), one_hot(accion, ACTION_SIZE)])
    input_tensor = torch.tensor([input_vec], dtype=torch.float32)

    prediccion = MODELO(input_tensor)
    estadoPredicho = torch.argmax(prediccion).item()

    # Almacenar estados y predicciones
    estadoReal.append(estado)
    estadosPredichos.append(estadoPredicho)
    accionesTomadas.append(accion)

    print( f"    Estado actual: {estado}, Acción: {accion} → Estado predicho: {estadoPredicho}" )

    # Visualizar el entorno
    entorno.render()

    # Realizar la acción en el entorno y obtener el siguiente estado
    estado, _, FIN, _, _ = entorno.step(accion)
    if FIN:
        estado, _ = entorno.reset()

#--------------------------------------------------------------------------
# 7. Mostramos el gráfico de las transiciones
#--------------------------------------------------------------------------
print( '6. Mostramos el gráfico de las transicciones en el entorno de Taxi-v3' )
plt.figure( figsize = ( 10, 6 ) )

# Mostramos gráficamente el estado real vs estado predicho
plt.subplot( 1, 2, 1 )
plt.plot( estadoReal, label = 'Estado Real', marker = 'o' )
plt.plot( estadosPredichos, label = 'Estado Predicho', marker = 'x' )
plt.title( "Estado Real vs Estado Predicho")
plt.xlabel( "Paso")
plt.ylabel( "Estado")
plt.legend()

# Mostramos gráficamente las acciones tomadas
plt.subplot( 1, 2, 2 )
plt.plot( accionesTomadas, label = 'Acciones Tomadas', marker = 's', color = 'r' )
plt.title( "Acciones Tomadas" )
plt.xlabel( "Paso" )
plt.ylabel( "Acción" )
plt.legend()

# Mostramos los gráficos
plt.tight_layout()
plt.show()

# Cerramos el entorno al final
entorno.close()

1. Recolectamos datos del entorno de Taxi-v3


100%|██████████| 500/500 [00:28<00:00, 17.84it/s]



2. Hacemos el preprocesamiento de los datos del entorno de Taxi-v3
